Try only use information before the harvesting season, add el nino effect as a feature, remove year, March-April is bud break, see if weather in Nov - April have a effect on harvest date, take out 2007, 2009 and 2010 as testing

In [1]:
%matplotlib inline  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
climate = pd.read_csv('climate.csv', sep='\t')
deliveries = pd.read_csv('deliveries.csv', sep='\t')

In [3]:
climate['crushdate'] = pd.to_datetime(climate['day'], format='%Y-%m-%d %H:%M:%S')
deliveries['crushdate'] = pd.to_datetime(deliveries['crushdate'], format='%m/%d/%y')

climate['year'] = climate['crushdate'].apply(lambda x: x.year).astype(int)
climate['month'] = climate['crushdate'].apply(lambda x: x.month).astype(int)
climate['day'] = climate['crushdate'].apply(lambda x: x.day).astype(int)

deliveries['year'] = deliveries['crushdate'].apply(lambda x: x.year).astype(int)
deliveries['month'] = deliveries['crushdate'].apply(lambda x: x.month).astype(int)
deliveries['day'] = deliveries['crushdate'].apply(lambda x: x.day).astype(int)

In [4]:
deliveries['tier'] = deliveries['programno'].apply(lambda x: x[:3])
deliveries['varietal'] = deliveries['programno'].apply(lambda x: x[3:6]).astype(int)
deliveries['origin'] = deliveries['programno'].apply(lambda x: x[6:])

In [5]:
# Look at one varietal first
v = deliveries[deliveries['varietal'] == 201]

In [6]:
v = v.sort_values(['crushdate', 'pblk'])
avg = v.groupby(['crushdate', 'pblk']).agg({'brix':np.mean, 'tons':np.mean})

v = v.drop(['brix', 'tons'], axis=1)
v = v.drop_duplicates(['crushdate', 'pblk'])

v['avg_brix'] = avg['brix'].tolist()
v['avg_tons'] = avg['tons'].tolist()

In [7]:
print(v.shape)
v.head()

(3993, 14)


,weighttagno,tagyear,siteno,programno,crushdate,pblk,year,month,day,tier,varietal,origin,avg_brix,avg_tons
106864,898323,1,2,BWR201CAL,2001-09-10,114695,2001,9,10,BWR,201,CAL,24.550000,1.544375
106855,898103,1,2,BWR201CAL,2001-09-10,117064,2001,9,10,BWR,201,CAL,25.200000,1.484088
106853,898012,1,2,BWR201CAL,2001-09-10,117079,2001,9,10,BWR,201,CAL,24.583333,1.421350
106923,898513,1,2,BWR201CAL,2001-09-11,114695,2001,9,11,BWR,201,CAL,24.668750,1.690106
106924,898520,1,2,BWR201CAL,2001-09-11,117064,2001,9,11,BWR,201,CAL,24.871428,1.732350


In [8]:
columns = ['crushdate', 'year', 'month', 'day', 'pblk', 'avg_brix', 'tier', 'origin']
v = v[columns]

In [9]:
v['pblk'].describe()

count      3993.000000
mean     122443.112948
std        8960.478489
min      100308.000000
25%      114802.000000
50%      118927.000000
75%      132318.000000
max      140100.000000
Name: pblk, dtype: float64

In [10]:
# Add more columns to the dataframe using the climate data
# Columns to be added:
# logitude, latitude of the pblk
# ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp of yesterday
# average ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for the past one week
# average ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for the past one month
# average ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for the past three month
# count of ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp that is greater than the mean for the past one week
# count of ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp that is greater than the mean for the past one month
# count of ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp that is greater than the mean for the past three month
# low rank approximation of ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for each pblk

In [11]:
# Add longitude, latitude
location = climate[['pblk', 'long', 'lat']]
location = location.drop_duplicates(['pblk', 'long', 'lat'])
v = pd.merge(v, location, how="left", on='pblk')
v.head()

,crushdate,year,month,day,pblk,avg_brix,tier,origin,long,lat
0,2001-09-10,2001,9,10,114695,24.550000,BWR,CAL,-8400.6517,2523.9900
1,2001-09-10,2001,9,10,117064,25.200000,BWR,CAL,-8411.7726,2546.2500
2,2001-09-10,2001,9,10,117079,24.583333,BWR,CAL,-8402.2106,2583.3066
3,2001-09-11,2001,9,11,114695,24.668750,BWR,CAL,-8400.6517,2523.9900
4,2001-09-11,2001,9,11,117064,24.871428,BWR,CAL,-8411.7726,2546.2500


In [12]:
# Add ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp of yesterday
features = ['ddays50', 'ddays50_97', 'ddays97', 'humidity', 'cloud_cover', 'pressure', 'maxtemp', 'mintemp']
yesterday_data = pd.DataFrame(columns=features)

for i in range(0, v.shape[0]):
    pblk = v.iloc[i].pblk
    yesterday = v.iloc[i].crushdate - timedelta(days=1)
    
    yesterday_data.loc[i] = climate[(climate['pblk'] == pblk) & 
                                    (climate['year'] == yesterday.year) & 
                                    (climate['month'] == yesterday.month) & 
                                    (climate['day'] == yesterday.day)][features].iloc[0].tolist()

In [13]:
for feature in features:
    v['yesterday_' + feature] = yesterday_data[feature]

In [14]:
v.head()

,crushdate,year,month,day,pblk,avg_brix,tier,origin,long,lat,yesterday_ddays50,yesterday_ddays50_97,yesterday_ddays97,yesterday_humidity,yesterday_cloud_cover,yesterday_pressure,yesterday_maxtemp,yesterday_mintemp
0,2001-09-10,2001,9,10,114695,24.550000,BWR,CAL,-8400.6517,2523.9900,20.13,20.13,0.0,0.59,0.12,1010.47,85.65,56.65
1,2001-09-10,2001,9,10,117064,25.200000,BWR,CAL,-8411.7726,2546.2500,21.78,21.78,0.0,0.58,0.05,1009.77,87.99,56.40
2,2001-09-10,2001,9,10,117079,24.583333,BWR,CAL,-8402.2106,2583.3066,20.49,20.49,0.0,0.62,0.01,1009.78,87.56,55.47
3,2001-09-11,2001,9,11,114695,24.668750,BWR,CAL,-8400.6517,2523.9900,19.12,19.12,0.0,0.62,0.08,1011.92,83.24,54.95
4,2001-09-11,2001,9,11,117064,24.871428,BWR,CAL,-8411.7726,2546.2500,20.66,20.66,0.0,0.61,0.10,1011.56,84.43,57.43


In [15]:
# Add average ddays50, ddays50_97, ddays97, humidity, cloud_cover, pressure, maxtemp, mintemp for the past one week
week_data = pd.DataFrame(columns=features)

for i in range(0, v.shape[0]):
    pblk = v.iloc[i].pblk
    yesterday = v.iloc[i].crushdate - timedelta(days=1)
    last_week = v.iloc[i].crushdate - timedelta(days=7)
    
    week_data.loc[i] = climate[(climate['pblk'] == pblk) & 
                               (climate['crushdate'] >= last_week) & 
                               (climate['crushdate'] <= yesterday)][features].mean().tolist()

In [16]:
week_data.head()

,ddays50,ddays50_97,ddays97,humidity,cloud_cover,pressure,maxtemp,mintemp
0,26.666667,25.343333,0.016667,0.426667,0.070000,1010.801667,93.251667,60.363333
1,28.166667,26.508333,0.025000,0.408333,0.093333,1010.378333,94.386667,61.881667
2,27.328333,25.308333,0.061667,0.423333,0.085000,1010.296667,94.895000,61.360000
3,24.820000,24.153333,0.013333,0.453333,0.071667,1010.756667,91.291667,58.840000
4,26.410000,25.410000,0.020000,0.433333,0.071667,1010.281667,92.770000,60.116667


In [17]:
for feature in features:
    v['week_' + feature] = week_data[feature]

In [18]:
month_data = pd.DataFrame(columns=features)

for i in range(0, v.shape[0]):
    pblk = v.iloc[i].pblk
    yesterday = v.iloc[i].crushdate - timedelta(days=1)
    last_month = v.iloc[i].crushdate - timedelta(days=31)
    
    month_data.loc[i] = climate[(climate['pblk'] == pblk) & 
                                (climate['crushdate'] >= last_month) & 
                                (climate['crushdate'] <= yesterday)][features].mean().tolist()

In [19]:
month_data.head()

,ddays50,ddays50_97,ddays97,humidity,cloud_cover,pressure,maxtemp,mintemp
0,26.985333,24.766000,0.064667,0.486667,0.068000,1011.789667,94.529667,60.482333
1,28.318333,25.211667,0.103667,0.475000,0.071667,1011.402333,95.417667,61.568000
2,27.569667,24.505333,0.127333,0.485000,0.046667,1011.335333,96.000667,60.881000
3,26.774333,24.555000,0.064667,0.488000,0.069000,1011.647667,94.253667,60.306333
4,28.121000,25.014333,0.103667,0.476667,0.072000,1011.251333,95.210667,61.369667


In [20]:
for feature in features:
    v['month_' + feature] = month_data[feature]

In [21]:
month3_data = pd.DataFrame(columns=features)

for i in range(0, v.shape[0]):
    pblk = v.iloc[i].pblk
    yesterday = v.iloc[i].crushdate - timedelta(days=1)
    last_month3 = v.iloc[i].crushdate - timedelta(days=93)
    
    month3_data.loc[i] = climate[(climate['pblk'] == pblk) & 
                               (climate['crushdate'] >= last_month3) & 
                               (climate['crushdate'] <= yesterday)][features].mean().tolist()

In [22]:
month3_data.head()

,ddays50,ddays50_97,ddays97,humidity,cloud_cover,pressure,maxtemp,mintemp
0,27.699565,24.991196,0.111739,0.472283,0.090109,1012.261087,94.189130,61.729674
1,29.115652,25.616957,0.177826,0.456304,0.098152,1011.876196,95.278913,62.921739
2,28.370109,24.986087,0.190978,0.465652,0.063478,1011.821957,95.888043,62.204348
3,27.635000,24.926630,0.111739,0.474783,0.091087,1012.230652,94.134891,61.694674
4,29.050435,25.551739,0.177826,0.459022,0.098261,1011.841196,95.249783,62.856087


In [23]:
for feature in features:
    v['month3_' + feature] = month3_data[feature]

In [32]:
v.to_csv('v201.csv', index=False)

In [25]:
v = pd.read_csv('v201.csv')
v['crushdate'] = pd.to_datetime(v['crushdate'], infer_datetime_format=True)

In [26]:
lr_ddays50 = pd.read_csv('lr_ddays50.csv')
lr_humidity = pd.read_csv('lr_humidity.csv')
lr_cloud_cover = pd.read_csv('lr_cloud_cover.csv')
lr_maxtemp = pd.read_csv('lr_maxtemp.csv')

lr_ddays50.head()

,ddays50_1,ddays50_2,ddays50_3,pblk
0,1611.355802,-12.218990,0.634748,100013
1,1367.716021,12.208003,-1.045634,100015
2,1370.836849,12.068548,-1.053069,100024
3,1727.012415,-16.562810,0.041056,100055
4,1346.071443,14.417660,-0.434827,100089


In [27]:
v = pd.merge(v, lr_ddays50, how="left", on='pblk')
v = pd.merge(v, lr_humidity, how="left", on='pblk')
v = pd.merge(v, lr_cloud_cover, how="left", on='pblk')
v = pd.merge(v, lr_maxtemp, how="left", on='pblk')

In [28]:
v.head()

,crushdate,year,month,day,pblk,avg_brix,tier,origin,long,lat,...,ddays50_3,humidity_1,humidity_2,humidity_3,cloud_cover_1,cloud_cover_2,cloud_cover_3,maxtemp_1,maxtemp_2,maxtemp_3
0,2001-09-10,2001,9,10,114695,24.550000,BWR,CAL,-8400.6517,2523.9900,...,0.767251,4.706198,53.969956,-0.579497,19.040687,-10.421925,13.830627,6883.758725,-7.165912,0.201284
1,2001-09-10,2001,9,10,117064,25.200000,BWR,CAL,-8411.7726,2546.2500,...,-0.704159,4.670784,53.727014,-0.890978,22.283158,-14.646745,26.571602,6911.038657,-8.000357,0.522319
2,2001-09-10,2001,9,10,117079,24.583333,BWR,CAL,-8402.2106,2583.3066,...,0.628670,4.815877,54.936938,-0.827228,24.389978,-16.436688,26.623762,6886.135793,-9.666395,-3.308100
3,2001-09-11,2001,9,11,114695,24.668750,BWR,CAL,-8400.6517,2523.9900,...,0.767251,4.706198,53.969956,-0.579497,19.040687,-10.421925,13.830627,6883.758725,-7.165912,0.201284
4,2001-09-11,2001,9,11,117064,24.871428,BWR,CAL,-8411.7726,2546.2500,...,-0.704159,4.670784,53.727014,-0.890978,22.283158,-14.646745,26.571602,6911.038657,-8.000357,0.522319


In [29]:
# Add number of days since the first delivery of the year
earliest_delivery = pd.DataFrame(columns=['year', 'earliest_date'])

crushdate = v[['crushdate', 'year']]
groups = crushdate.groupby('year')
i = 0
for year, df in groups:
    earliest_delivery.loc[i] = [int(year), df['crushdate'].min()]
    i += 1

In [30]:
earliest_delivery

,year,earliest_date
0,2001.0,2001-09-10
1,2002.0,2002-10-07
2,2003.0,2003-09-05
3,2004.0,2004-09-16
4,2005.0,2005-10-08
5,2006.0,2006-10-09
6,2007.0,2007-10-08
7,2008.0,2008-09-12
8,2009.0,2009-09-08
9,2010.0,2010-09-21


In [31]:
v = pd.merge(v, earliest_delivery, how='left', on='year')
v['time_diff'] = (v['crushdate'] - v['earliest_date']).astype('timedelta64[D]').astype(int)
v = v.drop(['day', 'earliest_date'], axis=1)

In [7]:
# Standardize continuous columns and create dummy for categorical
continuous = ['time_diff', 'long', 'lat', 'yesterday_ddays50', 'yesterday_ddays50_97', 'yesterday_ddays97',
              'yesterday_humidity', 'yesterday_cloud_cover', 'yesterday_pressure', 'yesterday_maxtemp',
              'yesterday_mintemp', 'week_ddays50', 'week_ddays50_97', 'week_ddays97', 'week_humidity',
              'week_cloud_cover', 'week_pressure', 'week_maxtemp', 'week_mintemp', 'month_ddays50',
              'month_ddays50_97', 'month_ddays97', 'month_humidity', 'month_cloud_cover', 'month_pressure',
              'month_maxtemp', 'month_mintemp', 'month3_ddays50', 'month3_ddays50_97', 'month3_ddays97',
              'month3_humidity', 'month3_cloud_cover', 'month3_pressure', 'month3_maxtemp', 'month3_mintemp',
              'ddays50_1', 'ddays50_2', 'ddays50_3', 'humidity_1', 'humidity_2', 'humidity_3', 'cloud_cover_1',
              'cloud_cover_2', 'cloud_cover_3', 'maxtemp_1', 'maxtemp_2', 'maxtemp_3']
categorical = ['year', 'month', 'tier', 'origin', 'which_week']

for item in categorical:
    dummy = pd.get_dummies(v[item], prefix=item)
    v = v.join(dummy.ix[:, :])
    v = v.drop(item, axis=1)

In [113]:
from sklearn import preprocessing

scalar = preprocessing.StandardScaler()
scalar.fit(v[continuous])
v[continuous] = scalar.transform(v[continuous])

In [114]:
index = v[['crushdate', 'pblk']]
y = v['avg_brix']
X = v.drop(['crushdate', 'pblk', 'avg_brix'], axis=1)

In [115]:
from sklearn.linear_model import Lasso

model = Lasso(alpha=0.01)
model.fit(X, y)
print('R^2 score: ', model.score(X, y))

R^2 score:  0.655285945274


In [116]:
coef = pd.DataFrame(columns=['feature', 'coef'])
coef['feature'] = X.columns.tolist()
coef['coef'] = model.coef_

coef = coef.sort_values('coef')
coef.head(30)

,feature,coef
97,tier_CHP,-2.199938
50,year_2005,-0.769009
113,origin_CAL,-0.574555
42,cloud_cover_3,-0.394675
48,year_2003,-0.327314
112,tier_VAL,-0.314855
51,year_2006,-0.314137
19,month_ddays50_97,-0.297610
28,month3_ddays97,-0.182880
5,yesterday_humidity,-0.124597


In [117]:
import statsmodels.api as sm

results = sm.OLS(y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               avg_brix   R-squared:                       0.703
Model:                            OLS   Adj. R-squared:                  0.697
Method:                 Least Squares   F-statistic:                     119.9
Date:                Sun, 22 Jan 2017   Prob (F-statistic):               0.00
Time:                        16:44:47   Log-Likelihood:                -7796.9
No. Observations:                6103   AIC:                         1.583e+04
Df Residuals:                    5984   BIC:                         1.663e+04
Df Model:                         118                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
long                     -0.3647      0.066     -5.568      0.000        -0.493    -0.236
lat                      -0.2314      0.072     -3.235      0.001        -0.372    -0.091
yesterday_ddays50         0.0685      0.083      0.827      0.408        -0.094     0.231
yesterday_ddays50_97     -0.1095      0.038     -2.917      0.004        -0.183    -0.036
yesterday_ddays97        -0.0579      0.027     -2.157      0.031        -0.111    -0.005
yesterday_humidity       -0.1244      0.023     -5.496      0.000        -0.169    -0.080
yesterday_cloud_cover    -0.0297      0.016     -1.883      0.060        -0.061     0.001
yesterday_pressure        0.0098      0.015      0.665      0.506        -0.019     0.039
yesterday_maxtemp         0.0640      0.045      1.431      0.152        -0.024     0.152
yesterday_mintemp         0.0667      0.036      1.832      0.067        -0.005     0.138
week_ddays50             -0.2003      0.172     -1.167      0.243        -0.537     0.136
week_ddays50_97          -0.1415      0.058     -2.450      0.014        -0.255    -0.028
week_ddays97             -0.0927      0.037     -2.474      0.013        -0.166    -0.019
week_humidity            -0.1334      0.032     -4.105      0.000        -0.197    -0.070
week_cloud_cover          0.0067      0.018      0.364      0.716        -0.029     0.043
week_pressure            -0.0545      0.021     -2.592      0.010        -0.096    -0.013
week_maxtemp              0.3094      0.086      3.605      0.000         0.141     0.478
week_mintemp              0.1462      0.081      1.803      0.071        -0.013     0.305
month_ddays50            -1.0738      0.319     -3.371      0.001        -1.698    -0.449
month_ddays50_97          0.2650      0.097      2.719      0.007         0.074     0.456
month_ddays97             0.1817      0.047      3.871      0.000         0.090     0.274
month_humidity           -0.0771      0.083     -0.928      0.354        -0.240     0.086
month_cloud_cover        -0.0769      0.033     -2.316      0.021        -0.142    -0.012
month_pressure            0.0978      0.037      2.613      0.009         0.024     0.171
month_maxtemp             0.4333      0.140      3.089      0.002         0.158     0.708
month_mintemp            -0.1039      0.150     -0.694      0.488        -0.397     0.190
month3_ddays50           -0.7599      0.325     -2.335      0.020        -1.398    -0.122
month3_ddays50_97        -0.1574      0.153     -1.026      0.305        -0.458     0.143
month3_ddays97           -0.1270      0.094     -1.350      0.177        -0.311     0.057
month3_humidity           0.1174      0.104      1.134      0.257        -0.086     0.320
month3_cloud_cover        0.0964      0.037      2.620      0.009         0.024     0.168
month3_pressure        